In [ ]:
!pip install -q mlflow nlp 2>/dev/null

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.1/575.1 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.4 MB/s eta 0:00:00


In [ ]:
!pip install datasets==2.21.0

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully uninstalled dill-0.3.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, 

In [ ]:
import pandas as pd
import gc
import re
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import torch
from transformers import AutoModel, AutoTokenizer
from transformers import TextDataset, LineByLineTextDataset, DataCollatorForLanguageModeling, \
pipeline, Trainer, TrainingArguments, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from nlp import Dataset
from imblearn.over_sampling import RandomOverSampler
import datasets
from transformers import pipeline
from bs4 import BeautifulSoup

from tqdm import tqdm
tqdm.pandas()

In [ ]:
# set parameters
train_fraction = 0.8 # fraction of a dataset used for training (the rest used for validation)
num_train_epochs = 2 # epochs to train
train_batch_size = 16 # batch size for training
eval_batch_size = 64 # batch size for validation
warmup_steps = 50
weight_decay = 0.02
BERT_MODEL = "bert-base-cased" # "distilbert-base-cased" # used distilbert cased version before
output_dir = "fake-news-classifier"

In [ ]:
# Mount Google Drive - applicable, if working on Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/WELFake_Dataset.csv').drop(['Unnamed: 0'], axis=1)
item0 = df.shape[0]
df = df.drop_duplicates()
item1 = df.shape[0]
print(f"There are {item0-item1} duplicates found in the dataset")
df['title'] = df['title'] + '\n\n\n' + df['text']
df = df[['title', 'label']]
df = df[~df['title'].isnull()]
print(df.shape)
df.sample(5).T


There are 8456 duplicates found in the dataset
(63121, 2)


,71361,11496,20292,38855,20416
title,Donald Trump Responds To Mockery Over Fake Sw...,Tweetwave\n\n\nThis time it's true no pantsTwe...,Rubio says prospect of Trump president is 'wor...,Trump lifts Cyber Command status to boost cybe...,The 7 Biggest Republican Lies About The Econo...
label,1,1,0,0,1


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 63121 entries, 0 to 72132
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   63121 non-null  object
 1   label   63121 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ MB


In [ ]:
df.tail(5)


,title,label
72127,WIKILEAKS EMAIL SHOWS CLINTON FOUNDATION FUNDS...,1
72129,Russians steal research on Trump in hack of U....,0
72130,WATCH: Giuliani Demands That Democrats Apolog...,1
72131,Migrants Refuse To Leave Train At Refugee Camp...,0
72132,Trump tussle gives unpopular Mexican leader mu...,0


In [ ]:
y = df[['label']]
df = df.drop(['label'], axis=1)
ros = RandomOverSampler(random_state=83)
df, y_resampled = ros.fit_resample(df, y)
del y
df['label'] = y_resampled
print(df.shape)
del y_resampled
gc.collect()


(69582, 2)


66

In [ ]:
df = df.sample(frac=1.0) # shuffle the data before train-test split
train_idx = int(df.shape[0]*train_fraction)
df_train = df[:train_idx]
df_test = df[train_idx:]
print(df_train['label'].mean(), df_test['label'].mean())


0.5002245576214857 0.4991018179205288


In [ ]:
del df
gc.collect()


0

In [ ]:
df_train = Dataset.from_pandas(df_train)
df_test = Dataset.from_pandas(df_test)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL, use_fast=True, low_cpu_mem_usage=False)


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["title"], truncation=True)

df_train = df_train.map(preprocess_function, batched=True)
df_test = df_test.map(preprocess_function, batched=True)


  0%|          | 0/56 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
df_train.remove_column_("title")
df_test.remove_column_("title")


In [ ]:
df_train

Dataset(features: {'label': Value(dtype='int64', id=None), '__index_level_0__': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 55665)

In [ ]:
df_test

Dataset(features: {'label': Value(dtype='int64', id=None), '__index_level_0__': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 13917)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
tokenizer.decode(df_train[0]['input_ids'])


'[CLS] Green groups sue for access to U. S. monument documents : lawsuit NEW YORK ( Reuters ) - The Sierra Club and five other U. S. environmental groups sued a White House advisory council and the U. S. Interior Department, saying they were illegally withholding information about a review of public land designations, according to court papers. In the lawsuit filed on Thursday, the groups asked the court to compel the Interior Department and the White House Council on Environmental Quality to provide documents on the department ’ s review of a group of national monuments, where industrial activities like mining and oil drilling are prohibited. Interior put more than a dozen monuments under review earlier this year. Several news outlets reported the department had determined which of those it would shrink, but it has not released a formal report. U. S. President Donald Trump, meanwhile, pledged last week to reduce the size of two monuments in Utah, according to U. S. Senator Orrin Hatch

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    BERT_MODEL, num_labels=2,
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False # Whether the model returns all hidden-states.
)

model.config.id2label = {0: 'FAKE', 1: 'REAL'}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model.num_parameters(only_trainable=True)/1e6)


108.31181


In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    logging_dir='./logs',
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    logging_strategy='steps',
    logging_first_step=True,
    load_best_model_at_end=True,
    logging_steps=1,
    evaluation_strategy='epoch',
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    eval_steps=1,
    save_strategy='epoch',
    save_total_limit=1, # save fewer checkpoints to limit used space
    report_to="mlflow",  # log to mlflow
)

# Define the trainer:
# instantiate the trainer class and check for available devices
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=df_train,
    eval_dataset=df_test,
    data_collator=data_collator
)


In [ ]:
trainer.evaluate()


{'eval_loss': 0.7437301278114319,
 'eval_model_preparation_time': 0.0074,
 'eval_accuracy': 0.5008981820794711,
 'eval_runtime': 421.0811,
 'eval_samples_per_second': 33.051,
 'eval_steps_per_second': 0.518}

In [ ]:
trainer.train()

2024/11/29 08:49:43 ERROR mlflow.utils.async_logging.async_logging_queue: Run Id 28fbbf08d4374b25bde0301bb4aea996: Failed to log run data: Exception: Changing param values is not allowed. Param with key='per_device_train_batch_size' was already logged with value='8' for run ID='28fbbf08d4374b25bde0301bb4aea996'. Attempted logging new value '16'.


Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy
1,0.000100,0.016590,0.007400,0.995904
2,0.000000,0.014023,0.007400,0.996838


TrainOutput(global_step=6960, training_loss=0.030700017334103366, metrics={'train_runtime': 11414.1758, 'train_samples_per_second': 9.754, 'train_steps_per_second': 0.61, 'total_flos': 2.929206540435864e+16, 'train_loss': 0.030700017334103366, 'epoch': 2.0})

In [ ]:
trainer.evaluate()


{'eval_loss': 0.014022902585566044,
 'eval_model_preparation_time': 0.0074,
 'eval_accuracy': 0.9968383990802615,
 'eval_runtime': 419.4044,
 'eval_samples_per_second': 33.183,
 'eval_steps_per_second': 0.52,
 'epoch': 2.0}

In [ ]:
output_dir = "/content/drive/My Drive/fake-news-classifier"  # Change this to your desired path
trainer.save_model(output_dir)


In [ ]:
tokenizer.save_vocabulary(save_directory=f"{output_dir}")

('/content/drive/MyDrive/fake-news-classifier/vocab.txt',)

Testing model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

output_dir = "/content/drive/My Drive/fake-news-classifier"
model = AutoModelForSequenceClassification.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

In [ ]:
import torch
texts = ["Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing",     # Fake
          "WATCH: George W. Bush Calls Out Trump For Supporting White Supremacy",               # Fake
          "U.S. lawmakers question businessman at 2016 Trump Tower meeting: sources",           # True
           "Trump administration issues new rules on U.S. visa waivers"                          # True
         ]

max_length = 512  # You can set this value depending on the model you're using

# Tokenize the input text with padding and truncation
inputs = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Perform prediction
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Convert logits to probabilities
probabilities = torch.softmax(logits, dim=-1)

# Get predicted classes
predicted_classes = torch.argmax(probabilities, dim=-1)

print(predicted_classes)


tensor([1, 1, 0, 0])


In [ ]:
# Assuming 0 = True, 1 = Fake
class_labels = {0: 'True', 1: 'Fake'}

# Map predicted class to label
predicted_labels = [class_labels[label.item()] for label in predicted_classes]
print(predicted_labels)


['Fake', 'Fake', 'True', 'True']
